In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
## Load the data
data=pd.read_csv("Churn_Modelling.csv")
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
### Drop irrelevant features
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
##Encode categorical (qualitative) variables that is we encode male 1, Female 0
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## Onehot encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## combine all the one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## save the encoders and scaler in wb i.e. write byte mode
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [12]:
## Here we will divide the dataset into independent and dependent features

##independent feature/input/predictors
##data.drop('Exited', axis=1) removes the Exited column from the dataset, which is the target variable we want to predict.
##axis=1 specifies that the column (as opposed to a row) is being removed.
##The resulting x will contain all columns except Exited, representing the independent features used for model training.
x=data.drop('Exited', axis=1) 

##dependent feature
##data['Exited'] selects only the Exited column, which is the value we're trying to predict in a classification problem.
y=data['Exited'] 

##Split the data in training and testing set
# test_size=0.2 specifies that 20% of the data will be allocated to the testing set, while 80% will be used for training the model
#random_state=42 is used to ensure reproducibility by controlling the randomness of the splitting. 
# It ensures that every time the code is run, the data is split in the same way.
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=42)

##Scale the features
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


In [13]:
x_train


array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [14]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [15]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [17]:
## Build our ANN Model 
## ReLU introduces non-linearity to the model by converting negative values to zero, which helps in training deep neural networks.
##sigmoid activation function squashes the output to a range between 0 and 1, making it useful for binary classification
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ##Hidden layer 1 connected with iput layer. input_shape=(x_train.shape[1],): Specifies the shape of the input data. 
    ##Here, x_train.shape[1] represents the number of features (or columns) in the training dataset.This is required for the first layer to know the input structure, 
    # and it only needs to be specified for the first layer.
    Dense(32,activation='relu'), ##Hidden layer 1
    Dense(1,activation='sigmoid')   ## output layer. Dense(1): This layer has 1 neuron, which is suitable for binary classification tasks where the output is either 0 or 1.
]

)

/Users/peteral/Documents/AI/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
## How many parameters are in the model 
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
## This is one option to compile the model
#Adam (Adaptive Moment Estimation)  is one of the most popular optimization algorithms used in training neural networks 
# because it combines the advantages of two other optimizers: AdaGrad and RMSProp.
##learning_rate=0.01: The learning rate controls how much the model’s weights are adjusted 
# in response to the error each time the model’s weights are updated.
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [20]:
## Another option to compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [33]:

##Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import keras
##creating a directory path for storing logs generated during the training of a machine learning model, often for use with TensorBoard, 
# a tool for visualizing metrics such as loss and accuracy during training
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
print("TensorBoard logs will be saved to:", log_dir)

TensorBoard logs will be saved to: logs/fit/20241012-162824


In [25]:
##Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [26]:
##Train the model 
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - accuracy: 0.8597 - loss: 0.3393 - val_accuracy: 0.8480 - val_loss: 0.3603
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.8585 - loss: 0.3465 - val_accuracy: 0.8635 - val_loss: 0.3424
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.8652 - loss: 0.3420 - val_accuracy: 0.8580 - val_loss: 0.3425
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.8576 - loss: 0.3365 - val_accuracy: 0.8575 - val_loss: 0.3474
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.8541 - loss: 0.3410 - val_accuracy: 0.8580 - val_loss: 0.3523
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.8647 - loss: 0.3315 - val_accuracy: 0.8565 - val_loss: 0.3539
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.8618 - loss: 0.3175 - val_accuracy: 0.8620 - val_loss: 0.3422
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.8655 - loss: 0

In [34]:
##save in h5 file which is compatible with keras
model.save('model.h5')

In [28]:
## Load Tensorboard Extension
%load_ext tensorboard

In [35]:
%tensorboard --logdir /logs/fit

In [ ]:
##Load the pickle file 